In [1]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
plt.rcParams.update({'font.size': 20})

from matplotlib.font_manager import FontProperties
font = FontProperties()
font.set_name('Times New Roman')
plt.rcParams.update({'font.family':'serif'}) 

import sys
sys.path.append( '../utils' )
from utils import cap_bank_vals,vdict,get_tid,get_phase,get_best_setting,get_max_width

## Parsing phase Scans

In [2]:
def parse_scans(board,datecut='2022-09-26 14'):
    directory = f"../PhaseScans/board_{board}"
    scanDirs = []
    for x in ['1_20']: #,'1_08','1_32']:
        scanDirs.extend(os.listdir(f"{directory}/voltage_{x}/"))

    df = pd.DataFrame({'timestamp':scanDirs})
    df['time'] = df['timestamp'].apply(lambda x: pd.to_datetime(x,format="%d%b_%H%M%S").replace(year=2022))

    df = df.loc[df.time>datecut]
    df.set_index('time',inplace=True)
    df.sort_index(inplace=True)

    df['TID'] = get_tid(df.index.values,board)
    # df=df.loc[df.TID>0]                                                                                                                                                                                   

    scan_strs = {
        'eRx': 'PhaseScan',
        'eTx': 'DelayScan',
    }
    for i,val in enumerate(cap_bank_vals):
        df_cap = df.copy()
        df_cap['capBank_val'] = val
        for voltage in ['1_20']: #,'1_08','1_32']:
            for scan in ['eRx','eTx']:
                df_cap[f'{scan}Scan_{voltage}'] = df_cap['timestamp'].apply(lambda x: get_phase(scan, f'{directory}/voltage_{voltage}/{x}/{scan}_{scan_strs[scan]}_CapSelect_{val}.csv'))
                df_cap[f'{scan}Best_{voltage}'] = df_cap[f'{scan}Scan_{voltage}'].apply(lambda x: get_best_setting(x))
                df_cap[f'{scan}MaxWidth_{voltage}'] = df_cap[f'{scan}Scan_{voltage}'].apply(lambda x: get_max_width(x)[0])
                df_cap[f'{scan}2ndMaxWidth_{voltage}'] = df_cap[f'{scan}Scan_{voltage}'].apply(lambda x: get_max_width(x)[1])

        if i==0:
            df_new = df_cap.copy()
        else:
            df_new = pd.concat([df_cap, df_new])

    # df_new = df_new.reset_index()                                                                                                                                                                         
    return df_new

In [3]:
df_scans = parse_scans('10')

In [4]:
df_scans.columns

Index(['timestamp', 'TID', 'capBank_val', 'eRxScan_1_20', 'eRxBest_1_20',
       'eRxMaxWidth_1_20', 'eRx2ndMaxWidth_1_20', 'eTxScan_1_20',
       'eTxBest_1_20', 'eTxMaxWidth_1_20', 'eTx2ndMaxWidth_1_20'],
      dtype='object')